# Linear Regression 

Linear regression is a statistical method used to model the relationship between a dependent variable (often denoted as $y$) and one or more independent variables (often denoted as $x$). The basic idea is to find the straight line that best fits the data points in a scatter plot.

The most common form is **simple linear regression**, which models two variables:

$$y = mx + b$$

where $y$ is the dependent variable, $x$ is the independent variable, $m$ is the slope, and $b$ is the intercept. 

Given a set of input data $\{(x_i, y_i)\}$, the goal of linear regression is to find the values of $m$ and $b$ that best fit the data.

The values of $m$ and $b$ are chosen to minimize the **sum of squared errors** (SSE) $\sum_i (y_i - \hat{y}_i)^2$.

Taking partial derivatives with respect to $m$ and $b$, setting them to 0, and solving yields:

$$m = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sum_i (x_i - \bar{x})^2},\quad b = \bar{y} - m\bar{x}$$

**Multiple linear regression** models the relationship between multiple independent variables and one dependent variable. The best-fit hyperplane is:

$$y = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_n x_n = X^\top W$$

## Code 
### Simple linear regression 
A basic implementation in Python using least squares:

In [ ]:
import numpy as np

class LinearRegression:
    def __init__(self):
        self.slope = None
        self.intercept = None

    def fit(self, X, y):
        X = np.asarray(X).ravel()
        y = np.asarray(y).ravel()
        n = len(X)
        x_mean = np.mean(X)
        y_mean = np.mean(y)
        numerator = 0.0
        denominator = 0.0
        for i in range(n):
            numerator += (X[i] - x_mean) * (y[i] - y_mean)
            denominator += (X[i] - x_mean) ** 2
        self.slope = numerator / denominator
        self.intercept = y_mean - self.slope * x_mean

    def predict(self, X):
        X = np.asarray(X).ravel()
        return [self.slope * x + self.intercept for x in X]


In [ ]:
X = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])
lr = LinearRegression()
lr.fit(X, y)
print(lr.slope)
print(lr.intercept)
y_pred = lr.predict(X)
print(y_pred)


### Vectorized (Normal Equation)

$$y = XW\quad\text{and}\quad W = (X^\top X)^{-1}X^\top y$$

We will add a bias column of ones to $X$ so that the first coefficient corresponds to the intercept.

In [ ]:
import numpy as np

class LinearRegressionVectorized:
    def __init__(self):
        self.W = None

    def fit(self, X, y):
        """
        X: (n, d) array, without bias column.
        y: (n,) array
        """
        X = np.asarray(X)
        y = np.asarray(y).ravel()
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])  # prepend bias column
        self.W = np.linalg.inv(Xb.T @ Xb) @ Xb.T @ y

    def predict(self, X):
        X = np.asarray(X)
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])  # prepend bias column in prediction too
        return Xb @ self.W


In [ ]:
# Create example input data
X = np.array([[2, 2], [4, 5], [7, 8]])
y = np.array([9, 17, 26])

# Fit vectorized model
lv = LinearRegressionVectorized()
lv.fit(X, y)
print(lv.W)  # expected ~ [3., 1., 2.]

# Make predictions on new data
X_new = np.array([[10, 11], [13, 14]])
y_pred = lv.predict(X_new)
print(y_pred)  # expected ~ [43., 55.]


### Improvements
Here are some improvements to make a gradient-descent implementation more robust:

1. **Input validation**: Ensure `X` and `y` have the same length and are not empty.
2. **Vectorization**: Use NumPy operations instead of Python loops where possible.
3. **Regularization**: Add L2 penalty on weights (often **excluding** the bias term).
4. **Gradient Descent**: Avoid matrix inversion on large data; learn via iterative updates.

In [ ]:
import numpy as np

class LinearRegressionGD:
    def __init__(self, regul=0.0):
        self.regul = float(regul)
        self.W = None

    def fit(self, X, y, lr=0.01, num_iter=1000, verbose=False):
        X = np.asarray(X)
        y = np.asarray(y).ravel()
        if len(X) != len(y) or len(X) == 0:
            raise ValueError("X and y must have the same length and cannot be empty")

        # Add bias term to X -> [1 X]
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])

        # Initialize W to zeros
        self.W = np.zeros(Xb.shape[1])

        for i in range(num_iter):
            # Predictions
            y_pred = Xb @ self.W

            # L2 regularization excluding bias term
            W_reg = self.W.copy()
            W_reg[0] = 0.0

            # Cost (MSE + L2), shown optionally
            cost = np.sum((y_pred - y) ** 2) + self.regul * np.sum(W_reg ** 2)

            # Gradient (d/dW of MSE + L2)
            gradients = 2 * (Xb.T @ (y_pred - y)) + 2 * self.regul * W_reg

            # Update rule
            self.W -= lr * gradients

            if verbose and (i % 1000 == 0 or i == num_iter - 1):
                print(f"iter={i}, cost={cost:.6f}")

    def predict(self, X):
        X = np.asarray(X)
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])
        return Xb @ self.W


### Test

In [ ]:
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 5, 4, 5])
gd = LinearRegressionGD(regul=0.1)
gd.fit(X, y, lr=0.01, num_iter=10000, verbose=True)
print(gd.W)
y_pred = gd.predict(X)
print(y_pred)


## Visualize

In [ ]:
import matplotlib.pyplot as plt

# Use the data from the GD example above
X_plot = X.ravel()
y_plot = y
y_pred_plot = y_pred

# Sort for a clean line
order = np.argsort(X_plot)
X_sorted = X_plot[order]
y_pred_sorted = y_pred_plot[order]

plt.scatter(X_plot, y_plot)
plt.plot(X_sorted, y_pred_sorted)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression')
plt.show()
